In [ ]:
def estimate_errors(vct_bits_tx, vct_bits_rx, vct_symbs_tx, vct_symbs_rx, pow_noise):
  bit_errors = 0; symb_errors = 0;
  ending_symbs = 10 # to avoid symbols (or bits) errors due to truncation
  lag = det_lag(vct_bits_tx[0:100], vct_bits_rx[0:100]) # lag = -2 # bits vectors lag is COD_LEN(=2) times symbols vectors lag

  vct_bits_tx = vct_bits_tx[TRAINING_END*COD_LEN : -ending_symbs*COD_LEN -1]
  vct_bits_rx = vct_bits_rx[TRAINING_END*COD_LEN : -ending_symbs*COD_LEN -1]
  vct_symbs_tx = vct_symbs_tx[TRAINING_END : -ending_symbs -1]
  vct_symbs_rx = vct_symbs_rx[TRAINING_END : -ending_symbs -1]

  if lag < 0: # synchronyze transmitted and received vectors
    vct_bits_tx = vct_bits_tx[abs(lag):-1]
    vct_symbs_tx = vct_symbs_tx[abs(lag)//2:-1] # abs(lag)//2 because lag is always even
  else:
    vct_bits_rx = vct_bits_rx[abs(lag):-1]
    vct_symbs_rx = vct_symbs_rx[abs(lag)//2:-1]
  if len(vct_bits_tx) > len(vct_bits_rx): # yielding vectors with the same length
    vct_bits_tx = vct_bits_tx[:len(vct_bits_rx)]
    vct_symbs_tx = vct_symbs_tx[:len(vct_symbs_rx)]
  if len(vct_bits_tx) < len(vct_bits_rx):
    vct_bits_rx = vct_bits_rx[:len(vct_bits_tx)]
    vct_symbs_rx = vct_symbs_rx[:len(vct_symbs_tx)]

  # with np.printoptions(threshold=np.inf):
  #   print('lag ')
  #   print(lag)
  #   print( ' vct_bits_tx ' )
  #   print(  vct_bits_tx  )
  #   print( ' vct_bits_rx ' )
  #   print(  vct_bits_rx  )
  #   print( ' vct_bits_tx[TRAINING_END*COD_LEN:-1][0:50] ' )
  #   print(  vct_bits_tx[TRAINING_END*COD_LEN:-1][0:50]  )
  #   print( ' vct_bits_rx[TRAINING_END*COD_LEN:-1][0:50] ' )
  #   print(  vct_bits_rx[TRAINING_END*COD_LEN:-1][0:50]  )
  #   input()

  #if pow_noise < 0.02: # Print for DEBUG
  #  output.clear() # clean console
  #  with np.printoptions(threshold=np.inf, linewidth=np.inf): # threshold prints all values in a large array, linewidth prints all in one line
  #    print('vct_bits_tx(I) & vct_bits_rx(II): ')
  #    print(np.array([vct_bits_tx, vct_bits_rx]), sep='\n') # print the two complete vectors in two rows
  #    #input()
  #    #!kill -9 -1 # stop program

  for it_b in np.arange(len(vct_bits_tx)): 
    if vct_bits_tx[it_b] != vct_bits_rx[it_b]: # bit error
      bit_errors += 1
  for it_s in np.arange(len(vct_symbs_tx)): 
    if vct_symbs_tx[it_s] != vct_symbs_rx[it_s]: # symbol error
      symb_errors += 1

  ber_point = bit_errors/len(vct_bits_tx)
  ser_point = symb_errors/len(vct_symbs_tx)

  return [ber_point, ser_point]